<a href="https://colab.research.google.com/github/NiloyPurkait/GSoC-2020/blob/master/RDF_GAN_discriminator_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import random
import pickle
import math
import os
import re
import unicodedata
from functools import reduce
import numpy as np
from tqdm import tqdm
import time

import tensorflow as tf
from tensorflow.keras import backend as K

import logging


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

#tiny number
_NEG_INF = -1e8

# Preprocessing Helper functions 


In [0]:



def _tensorize(vocab, text):
    """
    Function to convert texts into number sequences first, and then
    add padding. Basically, tensorising them.
    :param vocab: The vocab which is used to lookup ids
    :type vocab: tf.tokenizer obj
    :param text: A list of sentences or a text file
    :type text: list
    :return: tensorised text data
    :rtype: tf.tensor
    """
    tensor = vocab.texts_to_sequences(text)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                           padding='post')

    return tensor


In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:

def padding(tensor, max_length):
    """
    Pads the given tensor to a maximum sequence length along
    axis 1.
    for ex -
    let the tensor be [1,2,3,4] if th given max_length is 5
    the tensor becomes [1,2,34,0]
    Mostly used to pad the target sentences of the multilingual
    model and the node_list of all models,

    :param tensor:A tf tensor
    :type tensor:tf.tensor
    :param max_length:Dimension along axis 1, of the new tensor
    :type max_length:int
    :return:The padded tensor
    :rtype:tf tensor.
    """

    padding = tf.constant([[0, 0], [0, max_length - tensor.shape[1]]])
    padded_tensor = tf.pad(tensor, padding, mode='CONSTANT')

    return padded_tensor

In [0]:
# from src.DataLoader imports

## Dataset Loading Functions

In [0]:
def LoadGatDataset(train_path, eval_path, test_path, srv_vocab,
                   tgt_vocab, lang, num_examples=None):
    train_ = {}
    eval_ = {}
    test_ = {}


    # load the train and eval datasets
    with open(train_path, 'rb') as f:
        train_set = pickle.load(f)
    with open(eval_path, 'rb') as f:
        eval_set = pickle.load(f)
    with open(test_path, 'rb') as f:
        test_set = pickle.load(f)
    with open(srv_vocab, 'rb') as f:
        src_vocab = pickle.load(f)

    train_input, train_tgt = zip(*train_set)
    eval_input, eval_tgt = zip(*eval_set)
    (train_nodes, train_labels, train_node1, train_node2) = zip(*train_input)
    (eval_nodes, eval_labels, eval_node1, eval_node2) = zip(*eval_input)
    (test_nodes, test_labels, test_node1, test_node2) = zip(*test_set)

    train_["train_node_tensor"] = _tensorize(src_vocab, train_nodes)
    train_["train_label_tensor"] = _tensorize(src_vocab, train_labels)
    train_["train_node1_tensor"] = _tensorize(src_vocab, train_node1)
    train_["train_node2_tensor"] = _tensorize(src_vocab, train_node2)

    eval_["eval_node_tensor"] = _tensorize(src_vocab, eval_nodes)
    eval_["eval_label_tensor"] = _tensorize(src_vocab, eval_labels)
    eval_["eval_node1_tensor"] = _tensorize(src_vocab, eval_node1)
    eval_["eval_node2_tensor"] = _tensorize(src_vocab, eval_node2)

    test_["test_node_tensor"] = _tensorize(src_vocab, test_nodes)
    test_["test_label_tensor"] = _tensorize(src_vocab, test_labels)
    test_["test_node1_tensor"] = _tensorize(src_vocab, test_node1)
    test_["test_node2_tensor"] = _tensorize(src_vocab, test_node1)


    train_tgt_tensor = src_vocab.texts_to_sequences(train_tgt)
    train_["train_tgt_tensor"] = tf.keras.preprocessing.sequence.pad_sequences(train_tgt_tensor, padding='post')
    eval_tgt_tensor = src_vocab.texts_to_sequences(eval_tgt)
    eval_["eval_tgt_tensor"] = tf.keras.preprocessing.sequence.pad_sequences(eval_tgt_tensor, padding='post')
    target_vocab = src_vocab

    return (train_, eval_, test_, src_vocab, target_vocab, max_length(train_tgt_tensor))

In [0]:
def GetGATDataset(train_path, eval_path,
                  test_path, src_vocab,
                  tgt_vocab, lang,
                  set=None):
  

    (train, eval, test, src_vocab, tgt_vocab, max_length_targ) = LoadGatDataset(train_path,
                                                                                eval_path,
                                                                                test_path, src_vocab,
                                                                                tgt_vocab, lang)
    node_tensor = padding(train["train_node_tensor"], 16)
    label_tensor = padding(train["train_label_tensor"], 16)
    node1_tensor = padding(train["train_node1_tensor"], 16)
    node2_tensor = padding(train["train_node2_tensor"], 16)

    eval_nodes = padding(eval["eval_node_tensor"], 16)
    eval_labels = padding(eval["eval_label_tensor"], 16)
    eval_node1 = padding(eval["eval_node1_tensor"], 16)
    eval_node2 = padding(eval["eval_node2_tensor"], 16)

    test_nodes = padding(test["test_node_tensor"], 16)
    test_labels = padding(test["test_label_tensor"], 16)
    test_node1 = padding(test["test_node1_tensor"], 16)
    test_node2 = padding(test["test_node2_tensor"], 16)

    print('\nTrain Tensor shapes (nodes, labels, node1, node2, target) : ')
    print(node_tensor.shape, label_tensor.shape, node1_tensor.shape, node2_tensor.shape, train["train_tgt_tensor"].shape)
    print('\nEval Tensor shapes (nodes, labes, node1, node2) : ')
    print(eval_nodes.shape, eval_labels.shape, eval_node1.shape, eval_node2.shape, eval["eval_tgt_tensor"].shape)
    print('\nTest Tensor shapes (nodes, labes, node1, node2) : ')
    print(test_nodes.shape, test_labels.shape, test_node1.shape, test_node2.shape)

    TRAIN_BUFFER_SIZE = len(train["train_tgt_tensor"])
    EVAL_BUFFER_SIZE = len(eval["eval_tgt_tensor"])
    BATCH_SIZE = batch_size
    steps_per_epoch = len(train["train_tgt_tensor"]) // BATCH_SIZE
    src_vocab_size = len(src_vocab.word_index) + 1

    tgt_vocab_size = len(tgt_vocab.word_index) + 1

    dataset_size = train["train_tgt_tensor"].shape[0]

    dataset = tf.data.Dataset.from_tensor_slices((node_tensor, label_tensor,
                                                  node1_tensor, node2_tensor, train["train_tgt_tensor"])).shuffle(TRAIN_BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

    eval_set = tf.data.Dataset.from_tensor_slices((eval_nodes, eval_labels,
                                                   eval_node1, eval_node2, eval["eval_tgt_tensor"])).shuffle(EVAL_BUFFER_SIZE)
    eval_set = eval_set.batch(BATCH_SIZE, drop_remainder=True)

    test_set = tf.data.Dataset.from_tensor_slices((test_nodes, test_labels,
                                                   test_node1, test_node2))
    test_set = test_set.batch(BATCH_SIZE, drop_remainder=True)

    if set == None:
        return (dataset, eval_set, test_set, TRAIN_BUFFER_SIZE, BATCH_SIZE, steps_per_epoch,
                src_vocab_size, src_vocab, tgt_vocab_size, tgt_vocab,
                max_length_targ, dataset_size)
    elif set == 'test':
        return (test_set, TRAIN_BUFFER_SIZE, BATCH_SIZE, steps_per_epoch,
                src_vocab_size, src_vocab, tgt_vocab_size, tgt_vocab)


  



## Load processed Dataset

In [0]:




train_path = '/content/gdrive/My Drive/data/reif_train'
eval_path = '/content/gdrive/My Drive/data/reif_eval'
test_path = '/content/gdrive/My Drive/data/reif_test'
src_vocab = '/content/gdrive/My Drive/data/reif_src_vocab'
tgt_vocab =  '/content/gdrive/My Drive/data/train_vocab.model'


lang = 'eng'
batch_size = 64

In [132]:
(dataset, eval_set, test_set, BUFFER_SIZE, BATCH_SIZE, steps_per_epoch,
     src_vocab_size, src_vocab, tgt_vocab_size, tgt_vocab, max_length_targ, dataset_size) = GetGATDataset(train_path, eval_path,
                                                                                                     test_path, src_vocab,
                                                                                                      tgt_vocab, lang)


Train Tensor shapes (nodes, labels, node1, node2, target) : 
(34352, 16) (34352, 16) (34352, 16) (34352, 16) (34352, 82)

Eval Tensor shapes (nodes, labes, node1, node2) : 
(4316, 16) (4316, 16) (4316, 16) (4316, 16) (4316, 70)

Test Tensor shapes (nodes, labes, node1, node2) : 
(4224, 16) (4224, 16) (4224, 16) (4224, 16)


## Probe the loaded dataset

In [133]:
tgt_vocab_size, src_vocab_size

(10248, 10248)

In [134]:
 dataset_size, BUFFER_SIZE, BATCH_SIZE, steps_per_epoch

(34352, 34352, 64, 536)

## Make example input and target batches

In [0]:
#Example inputs and target batch for testing

nodes_, labels_, node1_, node2_, target_ = next(iter(dataset))

In [136]:
 def convert_to_string(target_, n):
    '''
    args:
      target_ : target tensor batch
      n = nth element in batch 

    usage:
      Takes in a tensor of (batch_size, vocab_size) and
      converts a batch instance into its string equivalent
    '''
    sequence = [tf.reduce_sum(i).numpy() for i in target_[n]]
    text = ' '.join([src_vocab.index_word[i] if i!=0 else '<pad>' for i in sequence])
    print(text)

convert_to_string(target_, 1)

<start> the comic character , auron s full name is lambien . <end> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


# Implement Encoder Module Layers:
To assemble the encoder modules, we must first define the following layers:
- Shared Embedding Layer
- Graph Attention Layer
- Feed Forward Layer


### Define Embedding Layer

# Implement LSTM model as Discriminator

In [0]:
from tensorflow.keras.layers import Input, Lambda,  Concatenate, LSTM
 
from tensorflow.keras.layers import Dense, Embedding,Activation, Dropout
from tensorflow.keras.models import Model

def Discriminator(V, E, H=64, dropout=0.1):
    '''
    Disciriminator model.
    # Arguments:
        V: int, Vocabrary size
        E: int, Embedding size
        H: int, LSTM hidden size
        dropout: float
    # Returns:
        discriminator: keras model
            input: word ids, shape = (B, T)
            output: probability of true data or not, shape = (B, 1)
    '''

    def Highway(x, num_layers=1, activation='relu', name_prefix=''):

      '''
      Layer wrapper function for Highway network
      # Arguments:
          x: tensor, shape = (B, input_size)
      # Optional Arguments:
          num_layers: int, dafault is 1, the number of Highway network layers
          activation: keras activation, default is 'relu'
          name_prefix: str, default is '', layer name prefix
      # Returns:
          out: tensor, shape = (B, input_size)
      '''
      input_size = K.int_shape(x)[1]
      for i in range(num_layers):
          gate_ratio_name = '{}Highway/Gate_ratio_{}'.format(name_prefix, i)
          fc_name = '{}Highway/FC_{}'.format(name_prefix, i)
          gate_name = '{}Highway/Gate_{}'.format(name_prefix, i)

          gate_ratio = Dense(input_size, activation='sigmoid', name=gate_ratio_name)(x)
          fc = Dense(input_size, activation=activation, name=fc_name)(x)
          x = Lambda(lambda args: args[0] * args[2] + args[1] * (1 - args[2]), name=gate_name)([fc, x, gate_ratio])
      return x


    input_ = Input(shape=(None,), dtype='int32', name='Input')   # (B, T)
    out = Embedding(V, E, mask_zero=True, name='Embedding')(input_)  # (B, T, E)
    out = LSTM(H)(out)
    out = Highway(out, num_layers=1)
    out = Dropout(dropout, name='Dropout')(out)
    out = Dense(1, activation='sigmoid', name='FC')(out)

    discriminator = Model(input_, out)
    return discriminator

# Define generator loss, Agent and Environment

## Define Agent class

## Initialize training variables and model

In [0]:
vocab_size = 10248
embedding_size = 16
BATCH_SIZE = batch_size
epochs = 10
step=0
steps = epochs * steps_per_epoch

In [0]:
## Generator checkpoint/not necessary for discriminator##################


checkpoint_dir = './gdrive/My Drive/RDF_GAN/training_checkpoints'




In [0]:
discriminator = Discriminator(vocab_size,
                              embedding_size,
                              H=64, dropout=0.1)
discriminator.compile('adam', 'binary_crossentropy')

## Define function to generate discriminator data

## Define Loss function

In [0]:
def save_progress(path):

  discriminator.save_weights(path+'/discriminator_weights')

In [0]:
def data_generator(real, targ_shape):

    #real_bool = np.array(real)[:,:]==0
    #real_len = [len(i) for i in real_bool]
    fake = np.random.randint(0, 10248, size=(batch_size, targ_shape))
    
    for ind, i in enumerate(fake):
      i[random.randint(0, 72):] = 0

    fake = np.concatenate([fake, np.ones((batch_size,1), dtype=np.int32)], axis=-1)

    disc_data = np.vstack([real, list(fake)])
    
    X, y = disc_data[:,:-1], disc_data[:,-1]

    return X, y

## Define training step

In [0]:

# Training step 

def train_step(discriminator, targ):


  # Store the target sequences as a negative class instance 
  disc_real = [[np.concatenate([np.array(y), np.array([1])])] for y in targ]
  


  # Get X_train, y_train for discriminator
  d_x, d_y = data_generator(disc_real[0], targ.shape[1])

  # Train Discriminator
  print('Discriminator loss :')
  discriminator.fit(d_x, d_y, batch_size=BATCH_SIZE, epochs=1)


  return discriminator

## Execute training loop

In [144]:
# Training algorithm



step=0
steps = epochs * steps_per_epoch
epochs=5
for epoch in range(epochs):

    with tqdm(total=(34352 // batch_size)) as pbar:
        for (batch, (node_, label_, node1_, node2_, targ_)) in tqdm(enumerate(dataset)):
            start = time.time()
            step += 1

            # Store the target sequences as a negative class instance 
            disc_real = [[np.concatenate([np.array(y), np.array([1])])] for y in targ_]

            # Get X_train, y_train for discriminator
            d_x, d_y = data_generator(disc_real[0], targ_.shape[1])

            # Train Discriminator
            print('Discriminator loss :')
            discriminator.fit(d_x, d_y, batch_size=BATCH_SIZE, epochs=1)

            save_progress(checkpoint_dir)

            print('Epoch {} Batch {} '.format(epoch, batch))

            print('Time {} \n'.format(time.time() - start))
            pbar.update(1)








  0%|          | 0/536 [00:00<?, ?it/s]






0it [00:00, ?it/s]

Discriminator loss :
2/2 [==============================] - 0s 26ms/step - loss: 0.6932








  0%|          | 1/536 [00:04<36:31,  4.10s/it]






1it [00:04,  4.08s/it]

Epoch 0 Batch 0 
Time 4.038578748703003 

Discriminator loss :
2/2 [==============================] - 0s 24ms/step - loss: 0.6858








  0%|          | 2/536 [00:04<26:10,  2.94s/it]






2it [00:04,  2.93s/it]

...

Epoch 0 Batch 55 
Time 0.30649542808532715 

Discriminator loss :
2/2 [==============================] - 0s 22ms/step - loss: 0.0118








 11%|█         | 57/536 [00:19<02:18,  3.47it/s]






57it [00:19,  3.46it/s]

Epoch 0 Batch 56 
Time 0.2664310932159424 

Discriminator loss :
2/2 [==============================] - 0s 30ms/step - loss: 0.0301









 11%|█         | 57/536 [00:19<02:44,  2.91it/s]


KeyboardInterrupt: ignored

In [0]:
BOS = tgt_vocab.word_index['<start>']
EOS = tgt_vocab.word_index['<end>']

BOS, EOS

